## Agent

This notebook demonstrates how to use Cua's Agent to run a workflow in a virtual sandbox on Apple Silicon Macs.

### Installation

In [1]:
!pip uninstall cua-agent[all]

zsh:1: no matches found: cua-agent[all]


In [ ]:
!pip install "cua-agent[all]"

# Or install individual agent loops:
# !pip install cua-agent[anthropic]
# !pip install cua-agent[omni]

In [ ]:
# If locally installed, use this instead:
import os

os.chdir('../libs/agent')
!poetry install
!poetry build

!pip uninstall cua-agent -y
!pip install ./dist/cua_agent-0.1.0-py3-none-any.whl --force-reinstall

## Initialize a Computer Agent

Computer allows you to run an agentic workflow in a virtual sandbox instances on Apple Silicon. Here's a basic example:

In [4]:
from agent import AgentLoop, LLMProvider
from computer import Computer

Similar to Computer, you can either use the async context manager pattern or initialize the ComputerAgent instance directly.

In [11]:
import os
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

Direct initialization:

In [ ]:
from datetime import datetime
import logging
from pathlib import Path
from agent.core.computer_agent import ComputerAgent, LLM

computer = Computer(verbosity=logging.INFO)

# Create agent with Anthropic loop and provider
agent = ComputerAgent(
        computer=computer,
        loop=AgentLoop.ANTHROPIC,
        # loop=AgentLoop.OMNI,
        model=LLM(provider=LLMProvider.ANTHROPIC, name="claude-3-7-sonnet-20250219"),
        # model=LLM(provider=LLMProvider.OPENAI, name="gpt-4.5-preview"),
        save_trajectory=True,
        trajectory_dir=str(Path("trajectories")),
        only_n_most_recent_images=3,
        verbosity=logging.INFO
    )

tasks = [
"""
Please help me with the following task:
1. Open Safari browser
2. Go to Wikipedia.org
3. Search for "Claude AI" 
4. Summarize the main points you find about Claude AI
"""
]

async with agent:
    for i, task in enumerate(tasks, 1):
        print(f"\nExecuting task {i}/{len(tasks)}: {task}")
        async for result in agent.run(task):
            print(result)
        print(f"Task {i} completed")


Or using the Omni Agentic Loop:

In [ ]:
from datetime import datetime
import logging
from pathlib import Path

computer = Computer(verbosity=logging.INFO)

# Create agent with Anthropic loop and provider
agent = ComputerAgent(
    computer=computer,
    # loop=AgentLoop.ANTHROPIC,
    loop=AgentLoop.OMNI,
    model=LLM(provider=LLMProvider.OPENAI, name="gpt-4.5-preview"),
    # model=LLM(provider=LLMProvider.ANTHROPIC, name="claude-3-7-sonnet-20250219"),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    only_n_most_recent_images=3,
    verbosity=logging.INFO,
)

tasks = [
"""
Please help me with the following task:
1. Open Safari browser
2. Go to Wikipedia.org
3. Search for "Claude AI" 
4. Summarize the main points you find about Claude AI
"""
]

async with agent:
    for i, task in enumerate(tasks, 1):
        print(f"\nExecuting task {i}/{len(tasks)}: {task}")
        async for result in agent.run(task):
            print(result)
        print(f"Task {i} completed")
